# Descent method — Steepest descent and conjugate gradient in Python

*Python implementation*

Let’s start with this equation and we want to solve for x:

$Ax = b $

The solution x the minimize the function below when A is **symmetric positive definite** (otherwise, x could be the maximum). It is because the gradient of f(x), ∇f(x) = Ax- b. And when Ax=b, ∇f(x)=0 and thus x is the minimum of the function.

$f(x) = \frac{1}{2}x^TAx-x^Tb$

In this article, I am going to show you two ways to find the solution x — method of Steepest Descent and method of Conjugate Gradient.

## Method of Steepest Descent


In [ ]:
import numpy as np
from numpy import linalg as LA

def is_pos_def(x):
    """check if a matrix is symmetric positive definite"""
    return np.all(np.linalg.eigvals(x) > 0)

def steepest_descent(A, b, x):
    """
    Solve Ax = b
    Parameter x: initial values
    """
    if (is_pos_def(A) == False) | (A != A.T).any():
        raise ValueError('Matrix A needs to be symmetric positive definite (SPD)')
    r = b - A @ x
    k = 0
    while LA.norm(r) > 1e-10 :
        p = r
        q = A @ p
        alpha = (p @ r) / (p @ q)
        x = x + alpha * p
        r = r - alpha * q
        k =+ 1

    return x

Now let’s use this `steepest_descent` function to calculate
$$\begin{bmatrix} 3 & 2 \\ 2 & 3 \end{bmatrix} X = \begin{bmatrix} -2 \\ 7 \end{bmatrix}$$

In [ ]:
A = np.array([[3, 2], [2, 3]])
b = np.array([-2, 7])
x0 = np.array([0, 0 ])

In [ ]:
%%time
steepest_descent(A, b, x0)

With the steepest_descent method, we get a value of (-4,5) and a wall time > 1 ms.

## Conjugate gradient method in Python

In [ ]:
def conjugate_gradient(A, b):
    if (is_pos_def(A) == False) | (A != A.T).any():
        raise ValueError('Matrix A needs to be symmetric positive definite (SPD)')
    r = b 
    k = 0
    x = np.zeros(A.shape[-1])
    while LA.norm(r) > 1e-10 :
        if k == 0:
            p = r
        else: 
            gamma = - (p @ A @ r)/(p @ A @ p)
            p = r + gamma * p
        alpha = (p @ r) / (p @ A @ p)
        x = x + alpha * p
        r = r - alpha * (A @ p)
        k =+ 1
    return x

In [ ]:
%%time
conjugate_gradient(A, b)

With the conjugate_gradient function, we got the same value (-4, 5) and wall time < 400 μs, which is a lot faster than the steepest descent.

## Visualizing steepest descent and conjugate gradient descent

In [ ]:
import numpy as np
import holoviews as hv
hv.extension('plotly')

def steepest_descent_store_result(A, b, x):
    x_steps = [x]
    y_steps = [0.5 * x @ A @ x - x @ b]
    
    if (is_pos_def(A) == False) | (A != A.T).any():
        raise ValueError('Matrix A needs to be symmetric positive definite (SPD)')
    r = b - A @ x
    k = 0
    while LA.norm(r) > 1e-10 :
        p = r
        q = A @ p
        alpha = (p @ r) / (p @ q)
        x = x + alpha * p
        r = r - alpha * q
        k =+ 1
        x_steps.append(x)
        y_steps.append(0.5 * x @ A @ x - x @ b)

    return x, x_steps, y_steps

def conjugate_gradient_store_result(A, b):
    if (is_pos_def(A) == False) | (A != A.T).any():
        raise ValueError('Matrix A needs to be symmetric positive definite (SPD)')
    r = b 
    k = 0
    x = np.zeros(A.shape[-1])
    x_steps = [x]
    y_steps = [0.5 * x @ A @ x - x @ b]
    while LA.norm(r) > 1e-10 :
        if k == 0:
            p = r
        else: 
            gamma = - (p @ A @ r)/(p @ A @ p)
            p = r + gamma * p
        alpha = (p @ r) / (p @ A @ p)
        x = x + alpha * p
        r = r - alpha * (A @ p)
        k =+ 1
        x_steps.append(x)
        y_steps.append(0.5 * x @ A @ x - x @ b)

    return x, x_steps, y_steps

def viz_descent(x_steps, y_steps):
    size = 50
    x1s = np.linspace(-6, 6, size)
    x2s = np.linspace(-6, 6, size)
    x1, x2  = np.meshgrid(x1s, x2s)
    Z = np.zeros((size, size))
    for i in range(size):
        for j in range(size):
            x = np.array([x1[i,j], x2[i,j]])
            Z[i,j] = 0.5 * x @ A @ x - x @ b
    surface = hv.Surface((x1s, x2s, Z)).opts(colorbar=True, width=700, height=700, cmap='Turbo',  alpha=.75)
    points= np.concatenate([np.stack(x_steps), np.array(y_steps)[:, np.newaxis]], axis=1)
    path = hv.Path3D(points).opts(width=700, height=700, color='black', line_width=1)
    return surface * path

# visualize steepest descent method
_, x_steps, y_steps = steepest_descent_store_result(A, b, x0)
viz_descent(x_steps, y_steps)



In [ ]:
# visualize conjugate gradient method
_, x_steps, y_steps = conjugate_gradient_store_result(A, b)
viz_descent(x_steps, y_steps)

Hope you enjoy this article. If you are interested in the math behind the two methods, check out this [article](https://sophiamyang.github.io/DS/optimization/descentmethod1.html).

By Sophia Yang on [September 11, 2020](https://medium.com/dsc-msit/descent-method-steepest-descent-and-conjugate-gradient-in-python-85aa4c4aac7b)